<a href="https://colab.research.google.com/github/Nov05/DS-Unit-1-Sprint-3-Statistical-Tests-and-Experiments/blob/master/module1-statistics-probability-and-inference/LS_DS_131_Statistics_Probability_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# modified by nov05 on 2019-06-10
# DS-Unit-1-Sprint-3-Statistical-Tests-and-Experiments
# module1-statistics-probability-and-inference/

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import ttest_1samp
from scipy.stats import ttest_ind, ttest_ind_from_stats, ttest_rel

%matplotlib inline
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 1 Sprint 3 Assignment 1*

# Apply the t-test to real data

Your assignment is to determine which issues have "statistically significant" differences between political parties in this [1980s congressional voting data](https://archive.ics.uci.edu/ml/datasets/Congressional+Voting+Records). The data consists of 435 instances (one for each congressperson), a class (democrat or republican), and 16 binary attributes (yes or no for voting for or against certain issues). Be aware - there are missing values!

Your goals:

1. Load and clean the data (or determine the best method to drop observations when running tests)
2. Using hypothesis testing, find an issue that democrats support more than republicans with p < 0.01
3. Using hypothesis testing, find an issue that republicans support more than democrats with p < 0.01
4. Using hypothesis testing, find an issue where the difference between republicans and democrats has p > 0.1 (i.e. there may not be much of a difference)

Note that this data will involve *2 sample* t-tests, because you're comparing averages across two groups (republicans and democrats) rather than a single group against a null hypothesis.

Stretch goals:

1. Refactor your code into functions so it's easy to rerun with arbitrary variables
2. Apply hypothesis testing to your personal project data (for the purposes of this notebook you can type a summary of the hypothesis you formed and tested)

In [0]:
### YOUR CODE STARTS HERE
data_url = "https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data"
names_url = "https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.names"

In [0]:
my_data = pd.read_csv(data_url)

In [7]:
print(my_data.shape)
my_data.head()

(434, 17)


,republican,n,y,n.1,y.1,y.2,y.3,n.2,n.3,n.4,y.4,?,y.5,y.6,y.7,n.5,y.8
0,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,?
1,democrat,?,y,y,?,y,y,n,n,n,n,y,n,y,y,n,n
2,democrat,n,y,y,n,?,y,n,n,n,n,y,n,y,n,n,y
3,democrat,y,y,y,n,y,y,n,n,n,n,y,?,y,y,y,y
4,democrat,n,y,y,n,y,y,n,n,n,n,n,n,y,y,y,y


In [0]:
!curl https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.names

   1. Class Name: 2 (democrat, republican)
   2. handicapped-infants: 2 (y,n)
   3. water-project-cost-sharing: 2 (y,n)
   4. adoption-of-the-budget-resolution: 2 (y,n)
   5. physician-fee-freeze: 2 (y,n)
   6. el-salvador-aid: 2 (y,n)
   7. religious-groups-in-schools: 2 (y,n)
   8. anti-satellite-test-ban: 2 (y,n)
   9. aid-to-nicaraguan-contras: 2 (y,n)  
   10. mx-missile: 2 (y,n)
   11. immigration: 2 (y,n)
   12. synfuels-corporation-cutback: 2 (y,n)
   13. education-spending: 2 (y,n)
   14. superfund-right-to-sue: 2 (y,n)
   15. crime: 2 (y,n)
   16. duty-free-exports: 2 (y,n)
   17. export-administration-act-south-africa: 2 (y,n)

In [57]:
my_data.columns = np.arange(1, 18).tolist()
my_data.columns.tolist()

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]

In [19]:
my_data.head(2)

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,?
1,democrat,?,y,y,?,y,y,n,n,n,n,y,n,y,y,n,n


In [28]:
my_data[1].value_counts()

democrat      267
republican    167
Name: 1, dtype: int64

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.assign.html

In [0]:
my_data.replace(to_replace='n', value=0, inplace=True)
my_data.replace(to_replace='y', value=1, inplace=True)
my_data.replace(to_replace='?', value=np.nan, inplace=True)

In [35]:
my_data.sample(5)

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
1,democrat,NaN,1.0,1.0,NaN,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
311,democrat,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0
344,republican,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0
420,democrat,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0
315,democrat,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0


In [0]:
dem = my_data[my_data[1]=='democrat']
rep = my_data[my_data[1]=='republican']

## 1-sample t-test example

In a 1-sample T-test we are testing the mean of one sample against a null hypothesis of our choosing.

The null hypothesis that we designate depends on how we have encoded our data and the kind of questions that we want to test.

If I have encoded votes as 0 for no and 1 for yes, I want to test Democratic support for an issue, and I use a null hypothesis of 0, then I am comparing Democrat voting support against a null hypothesis of no Democrat support at all for a given issue.

If I use a null hypothesis of .5 then I am comparing the democrat voting support against a null hypothesis of democrats being neither in favor or against a particular issue.

If I use a null hypothesis of 1 then I am comparing the democrat voting support against a null hypothesis of all democrats being favor of a particular issue.

Lets use the 0 and .5 null-hypotheses to test the significance of those particular claims. They're all valid questions to be asking, they're just posing a slightly different question --testing something different.

In [51]:
# Null hypotheses are half of the republicans support some issue.

for col in my_data.columns.tolist()[1:]:
  print(col, '\t', round(dem[col].mean(),2), '\t', 
        ttest_1samp(dem[col], 0.5, nan_policy='omit'))

2 	 0.6 	 Ttest_1sampResult(statistic=3.431373087696574, pvalue=0.000699612317167372)
3 	 0.5 	 Ttest_1sampResult(statistic=0.06454972243678961, pvalue=0.9485867005339235)
4 	 0.89 	 Ttest_1sampResult(statistic=19.859406568628835, pvalue=5.75931504660857e-54)
5 	 0.05 	 Ttest_1sampResult(statistic=-31.67705343439813, pvalue=6.796885728494356e-91)
6 	 0.22 	 Ttest_1sampResult(statistic=-11.016877548066462, pvalue=2.5007537432253433e-23)
7 	 0.48 	 Ttest_1sampResult(statistic=-0.7464459604122172, pvalue=0.45608033540995874)
8 	 0.77 	 Ttest_1sampResult(statistic=10.424565592705058, pvalue=1.8326900884510166e-21)
9 	 0.83 	 Ttest_1sampResult(statistic=14.13618595353591, pvalue=4.190313037098042e-34)
10 	 0.76 	 Ttest_1sampResult(statistic=9.470521640429526, pvalue=2.3590277159598606e-18)
11 	 0.47 	 Ttest_1sampResult(statistic=-0.9246846884388146, pvalue=0.35598081794452796)
12 	 0.51 	 Ttest_1sampResult(statistic=0.18751153526716705, pvalue=0.8514092819865365)
13 	 0.14 	 Ttest_1sampResu

## 2-sample t-test (for means) example

In [52]:
# Null hypotheses are two parties have same means of votes on some issue.

for col in my_data.columns.tolist()[1:]:
  print(col, '\t', round(dem[col].mean(),2), '\t', 
        round(rep[col].mean(),2), '\t', 
        ttest_ind(dem[col], rep[col], nan_policy='omit'))

2 	 0.6 	 0.19 	 Ttest_indResult(statistic=9.15392122841775, pvalue=2.4195550274149624e-18)
3 	 0.5 	 0.5 	 Ttest_indResult(statistic=-0.02491808700047811, pvalue=0.9801332440121653)
4 	 0.89 	 0.13 	 Ttest_indResult(statistic=23.12119107755175, pvalue=6.013425749068062e-77)
5 	 0.05 	 0.99 	 Ttest_indResult(statistic=-49.213324889131904, pvalue=1.0401154557088818e-176)
6 	 0.22 	 0.95 	 Ttest_indResult(statistic=-21.06444879561732, pvalue=1.3035583587179729e-67)
7 	 0.48 	 0.9 	 Ttest_indResult(statistic=-9.694471882817886, pvalue=3.415425248481181e-20)
8 	 0.77 	 0.24 	 Ttest_indResult(statistic=12.457095175493478, pvalue=1.6414204749130307e-30)
9 	 0.83 	 0.15 	 Ttest_indResult(statistic=17.9719921713269, pvalue=6.835532456916702e-54)
10 	 0.76 	 0.12 	 Ttest_indResult(statistic=16.371375965888596, pvalue=1.0255175625864558e-46)
11 	 0.47 	 0.55 	 Ttest_indResult(statistic=-1.677960615722372, pvalue=0.0940899398968142)
12 	 0.51 	 0.13 	 Ttest_indResult(statistic=8.293603989407588, 

## Stretch goals:

Refactor your code into functions so it's easy to return with arbitrary variables

Apply hypothesis testing to your personal project data (for the purposes of this notebook you can type a summary of the hypothesis you formed and tested)

In [60]:
def ttest_1samp_null(samples, hypothesis_mean):
  ttest = ttest_1samp(samples, hypothesis_mean, nan_policy='omit')
  return samples.mean(), ttest.statistic, ttest.pvalue

ttest_1samp_null(dem[2], 0.4)
ttest_1samp_null(dem[2], 0.6)

(0.6046511627906976, 0.15250547056429176, 0.8789079366662332)